# FLUX.1 - Génération d'Images Avancée

**Module :** 02-Images-Advanced  
**Niveau :** Intermédiaire/Avancé  
**Durée estimée :** 45 minutes  

## Introduction

**FLUX.1** est un modèle de génération d'images de pointe développé par Black Forest Labs (équipe fondatrice de Stable Diffusion). Il représente une avancée significative en termes de qualité et de fidélité au prompt.

### Variantes FLUX.1

| Variante | Licence | Caractéristiques | Utilisation |
|----------|---------|------------------|-------------|
| **FLUX.1-pro** | Propriétaire | Meilleure qualité, API uniquement | Production |
| **FLUX.1-dev** | Non-commercial | Haute qualité, LoRA supporté | Recherche |
| **FLUX.1-schnell** | Apache 2.0 | Ultra-rapide (4 steps), local | Prototypage |

### Architecture

```
┌─────────────────────────────────────────────────────────┐
│                    FLUX.1 Architecture                  │
├─────────────────────────────────────────────────────────┤
│   Text Prompt                                           │
│       ↓                                                 │
│   [T5-XXL Encoder] + [CLIP-L Encoder]                  │
│       ↓              ↓                                  │
│   ┌───────────────────────────────────┐                │
│   │   Multimodal DiT (Diffusion       │                │
│   │   Transformer with Flow Matching) │                │
│   │   - 12B parameters                │                │
│   │   - Rotary Position Embeddings    │                │
│   └───────────────────────────────────┘                │
│       ↓                                                 │
│   [VAE Decoder] → Output Image                         │
└─────────────────────────────────────────────────────────┘
```

## Prérequis

- Module 00-GenAI-Environment complété
- GPU avec 12GB+ VRAM (pour exécution locale)
- Ou clé API pour services cloud (fal.ai, Replicate)

In [1]:
# =============================================================================
# 1. CONFIGURATION ET IMPORTS
# =============================================================================

import os
import sys
import time
import json
import base64
import requests
from io import BytesIO
from pathlib import Path
from datetime import datetime
from typing import Optional, Dict, List, Tuple, Any, Union

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Chargement variables d'environnement
from dotenv import load_dotenv
load_dotenv("../.env")
load_dotenv("../00-GenAI-Environment/.env")

# Configuration
FAL_API_KEY = os.getenv("FAL_API_KEY")
REPLICATE_API_KEY = os.getenv("REPLICATE_API_TOKEN")
HF_TOKEN = os.getenv("HUGGINGFACE_TOKEN") or os.getenv("HF_TOKEN")

# Détection du mode d'exécution
USE_LOCAL = False  # Changé dynamiquement si GPU disponible
USE_API = bool(FAL_API_KEY or REPLICATE_API_KEY)

print("╔════════════════════════════════════════════════════╗")
print("║   FLUX.1 - Génération d'Images Avancée            ║")
print("╚════════════════════════════════════════════════════╝")
print(f"\n📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"\n🔑 Configuration API:")
print(f"   FAL.ai: {'✅ Configuré' if FAL_API_KEY else '❌ Non configuré'}")
print(f"   Replicate: {'✅ Configuré' if REPLICATE_API_KEY else '❌ Non configuré'}")
print(f"   HuggingFace: {'✅ Configuré' if HF_TOKEN else '❌ Non configuré'}")

╔════════════════════════════════════════════════════╗
║   FLUX.1 - Génération d'Images Avancée            ║
╚════════════════════════════════════════════════════╝

📅 Date: 2026-02-25 20:52:26

🔑 Configuration API:
   FAL.ai: ❌ Non configuré
   Replicate: ❌ Non configuré
   HuggingFace: ❌ Non configuré


In [2]:
# =============================================================================
# 2. DÉTECTION GPU ET CHARGEMENT DIFFUSERS (LOCAL)
# =============================================================================

try:
    import torch
    
    if torch.cuda.is_available():
        device = "cuda"
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"\n🎮 GPU Détecté: {gpu_name}")
        print(f"   VRAM: {gpu_memory:.1f} GB")
        
        # FLUX.1-schnell requiert ~12GB, dev ~24GB
        if gpu_memory >= 12:
            USE_LOCAL = True
            print(f"   ✅ Suffisant pour FLUX.1-schnell en local")
        else:
            print(f"   ⚠️ VRAM insuffisant pour FLUX local, utilisation API")
    else:
        device = "cpu"
        print("\n⚠️ Pas de GPU CUDA détecté - Mode API recommandé")
except ImportError:
    device = "cpu"
    print("\n⚠️ PyTorch non installé - Mode API uniquement")

# Tentative de chargement diffusers
flux_pipeline = None

if USE_LOCAL:
    try:
        from diffusers import FluxPipeline
        print("\n📦 Diffusers disponible pour FLUX local")
    except ImportError:
        print("\n⚠️ diffusers non installé: pip install diffusers transformers accelerate")
        USE_LOCAL = False

print(f"\n🔧 Mode d'exécution: {'Local (GPU)' if USE_LOCAL else 'API Cloud'}")


⚠️ Pas de GPU CUDA détecté - Mode API recommandé

🔧 Mode d'exécution: API Cloud


In [3]:
# =============================================================================
# 3. CLIENT FLUX.1 UNIFIÉ (Local + API)
# =============================================================================

class FluxClient:
    """
    Client unifié pour FLUX.1 supportant:
    - Exécution locale via diffusers
    - API fal.ai
    - API Replicate
    """
    
    def __init__(self, mode: str = "auto"):
        """
        Args:
            mode: "local", "fal", "replicate", ou "auto" (détection automatique)
        """
        self.mode = mode
        self.pipeline = None
        
        if mode == "auto":
            if USE_LOCAL:
                self.mode = "local"
            elif FAL_API_KEY:
                self.mode = "fal"
            elif REPLICATE_API_KEY:
                self.mode = "replicate"
            else:
                raise ValueError("Aucun backend disponible. Configurez une API ou installez diffusers.")
        
        print(f"🔌 FluxClient initialisé en mode: {self.mode}")
    
    def load_local_model(self, model_id: str = "black-forest-labs/FLUX.1-schnell"):
        """Charge le modèle FLUX localement (GPU requis)."""
        if self.mode != "local":
            print("⚠️ Mode local non disponible")
            return
        
        from diffusers import FluxPipeline
        import torch
        
        print(f"\n📥 Chargement de {model_id}...")
        print("   (Cela peut prendre plusieurs minutes au premier lancement)")
        
        self.pipeline = FluxPipeline.from_pretrained(
            model_id,
            torch_dtype=torch.bfloat16,
            token=HF_TOKEN
        )
        self.pipeline.to("cuda")
        
        # Optimisations mémoire
        self.pipeline.enable_model_cpu_offload()
        
        print("   ✅ Modèle chargé")
    
    def generate(self, prompt: str, 
                 width: int = 1024, height: int = 1024,
                 num_inference_steps: int = 4,
                 guidance_scale: float = 0.0,
                 seed: int = None,
                 num_images: int = 1) -> List[Image.Image]:
        """
        Génère des images avec FLUX.1.
        
        Args:
            prompt: Description de l'image
            width, height: Dimensions (multiples de 8)
            num_inference_steps: Nombre d'étapes (schnell: 1-4, dev: 20-50)
            guidance_scale: CFG (0 pour schnell, 3.5 pour dev)
            seed: Graine pour reproductibilité
            num_images: Nombre d'images à générer
        
        Returns:
            Liste d'images PIL
        """
        if seed is None:
            seed = np.random.randint(0, 2**32)
        
        if self.mode == "local":
            return self._generate_local(prompt, width, height, num_inference_steps, 
                                        guidance_scale, seed, num_images)
        elif self.mode == "fal":
            return self._generate_fal(prompt, width, height, num_inference_steps,
                                      guidance_scale, seed, num_images)
        elif self.mode == "replicate":
            return self._generate_replicate(prompt, width, height, num_inference_steps,
                                            guidance_scale, seed, num_images)
        else:
            raise ValueError(f"Mode inconnu: {self.mode}")
    
    def _generate_local(self, prompt, width, height, steps, guidance, seed, num_images):
        """Génération locale avec diffusers."""
        import torch
        
        if self.pipeline is None:
            self.load_local_model()
        
        generator = torch.Generator("cuda").manual_seed(seed)
        
        print(f"\n🎨 Génération locale (seed: {seed})...")
        start = time.time()
        
        result = self.pipeline(
            prompt=prompt,
            width=width,
            height=height,
            num_inference_steps=steps,
            guidance_scale=guidance,
            generator=generator,
            num_images_per_prompt=num_images
        )
        
        elapsed = time.time() - start
        print(f"   ✅ Généré en {elapsed:.1f}s")
        
        return result.images
    
    def _generate_fal(self, prompt, width, height, steps, guidance, seed, num_images):
        """Génération via fal.ai API."""
        print(f"\n🌐 Génération via fal.ai (seed: {seed})...")
        
        headers = {
            "Authorization": f"Key {FAL_API_KEY}",
            "Content-Type": "application/json"
        }
        
        # Endpoint pour FLUX.1-schnell
        url = "https://fal.run/fal-ai/flux/schnell"
        
        payload = {
            "prompt": prompt,
            "image_size": {"width": width, "height": height},
            "num_inference_steps": steps,
            "seed": seed,
            "num_images": num_images
        }
        
        start = time.time()
        resp = requests.post(url, headers=headers, json=payload, timeout=120)
        
        if resp.status_code != 200:
            raise Exception(f"fal.ai error: {resp.text}")
        
        result = resp.json()
        elapsed = time.time() - start
        
        # Télécharger les images
        images = []
        for img_data in result.get("images", []):
            img_url = img_data.get("url")
            if img_url:
                img_resp = requests.get(img_url)
                images.append(Image.open(BytesIO(img_resp.content)))
        
        print(f"   ✅ Généré en {elapsed:.1f}s ({len(images)} images)")
        return images
    
    def _generate_replicate(self, prompt, width, height, steps, guidance, seed, num_images):
        """Génération via Replicate API."""
        print(f"\n🌐 Génération via Replicate (seed: {seed})...")
        
        headers = {
            "Authorization": f"Token {REPLICATE_API_KEY}",
            "Content-Type": "application/json"
        }
        
        # Créer la prédiction
        url = "https://api.replicate.com/v1/predictions"
        
        payload = {
            "version": "schnell",  # ou "dev" pour FLUX.1-dev
            "input": {
                "prompt": prompt,
                "width": width,
                "height": height,
                "num_inference_steps": steps,
                "guidance_scale": guidance,
                "seed": seed,
                "num_outputs": num_images
            }
        }
        
        start = time.time()
        resp = requests.post(url, headers=headers, json=payload)
        
        if resp.status_code not in [200, 201]:
            raise Exception(f"Replicate error: {resp.text}")
        
        prediction = resp.json()
        prediction_id = prediction["id"]
        
        # Polling pour attendre le résultat
        status_url = f"https://api.replicate.com/v1/predictions/{prediction_id}"
        while True:
            status_resp = requests.get(status_url, headers=headers)
            status = status_resp.json()
            
            if status["status"] == "succeeded":
                break
            elif status["status"] == "failed":
                raise Exception(f"Generation failed: {status.get('error')}")
            
            time.sleep(1)
        
        elapsed = time.time() - start
        
        # Télécharger les images
        images = []
        for img_url in status.get("output", []):
            img_resp = requests.get(img_url)
            images.append(Image.open(BytesIO(img_resp.content)))
        
        print(f"   ✅ Généré en {elapsed:.1f}s ({len(images)} images)")
        return images


# Instanciation du client
try:
    flux = FluxClient(mode="auto")
except ValueError as e:
    print(f"\n⚠️ {e}")
    print("\nPour utiliser ce notebook, configurez:")
    print("  - FAL_API_KEY dans .env (recommandé)")
    print("  - Ou REPLICATE_API_TOKEN dans .env")
    print("  - Ou installez diffusers avec un GPU 12GB+")
    flux = None


⚠️ Aucun backend disponible. Configurez une API ou installez diffusers.

Pour utiliser ce notebook, configurez:
  - FAL_API_KEY dans .env (recommandé)
  - Ou REPLICATE_API_TOKEN dans .env
  - Ou installez diffusers avec un GPU 12GB+


## 4. Génération de Base avec FLUX.1-schnell

FLUX.1-schnell est optimisé pour la rapidité (1-4 steps) tout en maintenant une bonne qualité.

In [4]:
# =============================================================================
# 4. GÉNÉRATION DE BASE
# =============================================================================

if flux:
    # Prompt de démonstration
    demo_prompt = """
    A serene Japanese zen garden at golden hour, 
    carefully raked sand patterns, moss-covered stones, 
    a small wooden bridge over a koi pond, 
    cherry blossoms falling gently, 
    photorealistic, 8k, masterpiece
    """.strip()
    
    print(f"\n📝 Prompt: {demo_prompt[:80]}...")
    
    # Génération avec FLUX.1-schnell (4 steps)
    images = flux.generate(
        prompt=demo_prompt,
        width=1024,
        height=1024,
        num_inference_steps=4,  # schnell: 1-4 steps
        guidance_scale=0.0,     # schnell: pas de guidance
        seed=42
    )
    
    if images:
        plt.figure(figsize=(10, 10))
        plt.imshow(images[0])
        plt.title("FLUX.1-schnell (4 steps)", fontsize=14)
        plt.axis('off')
        plt.tight_layout()
        plt.show()
        
        demo_image = images[0]
        print(f"\n📐 Dimensions: {demo_image.size}")
else:
    print("⚠️ Client FLUX non initialisé")

⚠️ Client FLUX non initialisé


## 5. Analyse du Nombre d'Étapes (Inference Steps)

Comparons l'impact du nombre d'étapes sur la qualité de génération.

In [5]:
# =============================================================================
# 5. ANALYSE DES INFERENCE STEPS
# =============================================================================

if flux:
    test_prompt = "A majestic wolf standing on a snowy mountain peak, aurora borealis in the sky, cinematic lighting"
    step_values = [1, 2, 4, 8]  # Pour schnell
    fixed_seed = 12345
    
    print(f"\n📊 Analyse Inference Steps")
    print(f"Prompt: '{test_prompt[:50]}...'")
    print(f"Valeurs testées: {step_values}")
    
    step_results = []
    
    for steps in step_values:
        print(f"\n--- Steps = {steps} ---")
        
        images = flux.generate(
            prompt=test_prompt,
            width=768,
            height=768,
            num_inference_steps=steps,
            guidance_scale=0.0,
            seed=fixed_seed
        )
        
        if images:
            step_results.append((steps, images[0]))
    
    # Affichage comparatif
    if step_results:
        fig, axes = plt.subplots(1, len(step_results), figsize=(16, 5))
        
        for i, (steps, img) in enumerate(step_results):
            axes[i].imshow(img)
            axes[i].set_title(f"Steps = {steps}", fontsize=12)
            axes[i].axis('off')
        
        plt.suptitle("Impact du nombre d'étapes (FLUX.1-schnell)", fontsize=14)
        plt.tight_layout()
        plt.show()
        
        print("\n📈 Observations:")
        print("   1 step:  Très rapide, structure de base")
        print("   2 steps: Bon compromis vitesse/qualité")
        print("   4 steps: Qualité optimale pour schnell")
        print("   8 steps: Diminishing returns, peu d'amélioration")
else:
    print("⚠️ Client FLUX non initialisé")

⚠️ Client FLUX non initialisé


## 6. Exploration des Ratios d'Aspect

FLUX.1 supporte différents ratios d'aspect pour des compositions variées.

In [6]:
# =============================================================================
# 6. RATIOS D'ASPECT
# =============================================================================

if flux:
    # Différents ratios
    aspect_ratios = [
        ("1:1 (Carré)", 1024, 1024),
        ("16:9 (Paysage)", 1024, 576),
        ("9:16 (Portrait)", 576, 1024),
        ("4:3 (Standard)", 1024, 768),
    ]
    
    aspect_prompt = "A stunning sunset over the ocean, vibrant orange and purple sky, silhouette of palm trees, photorealistic"
    
    print(f"\n🖼️ Exploration des Ratios d'Aspect")
    print(f"Prompt: '{aspect_prompt[:50]}...'")
    
    aspect_results = []
    
    for name, w, h in aspect_ratios:
        print(f"\n--- {name} ({w}x{h}) ---")
        
        images = flux.generate(
            prompt=aspect_prompt,
            width=w,
            height=h,
            num_inference_steps=4,
            seed=7777
        )
        
        if images:
            aspect_results.append((name, images[0]))
    
    # Affichage
    if aspect_results:
        fig = plt.figure(figsize=(16, 8))
        
        for i, (name, img) in enumerate(aspect_results):
            ax = fig.add_subplot(2, 2, i+1)
            ax.imshow(img)
            ax.set_title(name, fontsize=11)
            ax.axis('off')
        
        plt.suptitle("Comparaison des Ratios d'Aspect", fontsize=14)
        plt.tight_layout()
        plt.show()
else:
    print("⚠️ Client FLUX non initialisé")

⚠️ Client FLUX non initialisé


## 7. Batch Generation: Variations d'un Même Prompt

Générons plusieurs variations en changeant uniquement la seed.

In [7]:
# =============================================================================
# 7. BATCH GENERATION - VARIATIONS
# =============================================================================

if flux:
    variation_prompt = """
    A mystical forest with bioluminescent plants, 
    glowing mushrooms, fireflies, magical atmosphere, 
    fantasy art style, highly detailed
    """.strip()
    
    num_variations = 4
    base_seed = 1000
    
    print(f"\n🎲 Génération de {num_variations} variations")
    print(f"Prompt: '{variation_prompt[:50]}...'")
    
    variations = []
    
    for i in range(num_variations):
        seed = base_seed + i * 1000
        print(f"\n[{i+1}/{num_variations}] Seed: {seed}")
        
        images = flux.generate(
            prompt=variation_prompt,
            width=768,
            height=768,
            num_inference_steps=4,
            seed=seed
        )
        
        if images:
            variations.append((seed, images[0]))
    
    # Affichage grille
    if variations:
        fig, axes = plt.subplots(2, 2, figsize=(12, 12))
        axes = axes.flatten()
        
        for i, (seed, img) in enumerate(variations):
            axes[i].imshow(img)
            axes[i].set_title(f"Seed: {seed}", fontsize=11)
            axes[i].axis('off')
        
        plt.suptitle("Variations avec différentes seeds", fontsize=14)
        plt.tight_layout()
        plt.show()
        
        print(f"\n✅ {len(variations)} variations générées")
else:
    print("⚠️ Client FLUX non initialisé")

⚠️ Client FLUX non initialisé


## 8. Techniques de Prompt Engineering pour FLUX

FLUX.1 répond particulièrement bien à certains styles de prompts.

In [8]:
# =============================================================================
# 8. PROMPT ENGINEERING AVANCÉ
# =============================================================================

# Templates de prompts efficaces pour FLUX
prompt_templates = {
    "photorealistic": {
        "prefix": "A photorealistic image of",
        "suffix": ", shot with a Canon EOS R5, 85mm f/1.4, natural lighting, 8k resolution",
        "example": "a woman in a red dress walking through autumn leaves"
    },
    "cinematic": {
        "prefix": "Cinematic still from a movie,",
        "suffix": ", dramatic lighting, anamorphic lens, film grain, color graded",
        "example": "a detective in a noir city at night under rain"
    },
    "illustration": {
        "prefix": "Digital illustration of",
        "suffix": ", artstation trending, vibrant colors, highly detailed, concept art",
        "example": "a steampunk airship flying over a Victorian city"
    },
    "anime": {
        "prefix": "Anime artwork of",
        "suffix": ", Studio Ghibli style, beautiful scenery, soft colors, detailed",
        "example": "a young adventurer discovering a hidden temple in the forest"
    }
}

if flux:
    print("\n🎨 Comparaison des Styles de Prompts")
    print("=" * 50)
    
    style_results = []
    fixed_seed = 9999
    
    for style_name, template in prompt_templates.items():
        full_prompt = f"{template['prefix']} {template['example']}{template['suffix']}"
        
        print(f"\n--- Style: {style_name.upper()} ---")
        print(f"   Prompt: {full_prompt[:60]}...")
        
        images = flux.generate(
            prompt=full_prompt,
            width=768,
            height=768,
            num_inference_steps=4,
            seed=fixed_seed
        )
        
        if images:
            style_results.append((style_name, images[0]))
    
    # Affichage
    if style_results:
        fig, axes = plt.subplots(2, 2, figsize=(14, 14))
        axes = axes.flatten()
        
        for i, (style, img) in enumerate(style_results):
            axes[i].imshow(img)
            axes[i].set_title(f"Style: {style.capitalize()}", fontsize=12)
            axes[i].axis('off')
        
        plt.suptitle("Comparaison des Styles de Prompt", fontsize=14)
        plt.tight_layout()
        plt.show()
else:
    print("⚠️ Client FLUX non initialisé")
    print("\nVoici les templates de prompt recommandés pour FLUX:")
    for style, template in prompt_templates.items():
        print(f"\n{style.upper()}:")
        print(f"  Préfixe: {template['prefix']}")
        print(f"  Suffixe: {template['suffix']}")

⚠️ Client FLUX non initialisé

Voici les templates de prompt recommandés pour FLUX:

PHOTOREALISTIC:
  Préfixe: A photorealistic image of
  Suffixe: , shot with a Canon EOS R5, 85mm f/1.4, natural lighting, 8k resolution

CINEMATIC:
  Préfixe: Cinematic still from a movie,
  Suffixe: , dramatic lighting, anamorphic lens, film grain, color graded

ILLUSTRATION:
  Préfixe: Digital illustration of
  Suffixe: , artstation trending, vibrant colors, highly detailed, concept art

ANIME:
  Préfixe: Anime artwork of
  Suffixe: , Studio Ghibli style, beautiful scenery, soft colors, detailed


## 9. Génération de Texte dans les Images

FLUX.1 excelle particulièrement dans le rendu de texte lisible dans les images.

In [9]:
# =============================================================================
# 9. GÉNÉRATION DE TEXTE DANS LES IMAGES
# =============================================================================

if flux:
    text_prompts = [
        'A vintage neon sign that says "OPEN 24 HOURS" glowing in the night, realistic',
        'A birthday cake with elegant cursive text reading "Happy Birthday Sarah" in icing',
        'A wooden street sign pointing right with the text "Adventure Awaits" carved into it',
        'A coffee cup with "Good Morning!" written in latte art'
    ]
    
    print("\n✍️ Test de Génération de Texte")
    print("FLUX.1 est reconnu pour sa capacité à générer du texte lisible.")
    
    text_results = []
    
    for i, prompt in enumerate(text_prompts):
        print(f"\n[{i+1}/{len(text_prompts)}] {prompt[:50]}...")
        
        images = flux.generate(
            prompt=prompt,
            width=768,
            height=768,
            num_inference_steps=4,
            seed=2024 + i
        )
        
        if images:
            text_results.append((prompt.split('"')[1] if '"' in prompt else prompt[:20], images[0]))
    
    # Affichage
    if text_results:
        fig, axes = plt.subplots(2, 2, figsize=(12, 12))
        axes = axes.flatten()
        
        for i, (text, img) in enumerate(text_results):
            axes[i].imshow(img)
            axes[i].set_title(f'Texte: "{text}"', fontsize=10)
            axes[i].axis('off')
        
        plt.suptitle("Capacité de Génération de Texte", fontsize=14)
        plt.tight_layout()
        plt.show()
else:
    print("⚠️ Client FLUX non initialisé")

⚠️ Client FLUX non initialisé


## 10. Exercices Pratiques

### Exercice 1: Portrait Professionnel
Utilisez le template "photorealistic" pour générer un portrait professionnel d'une personne de votre choix.

### Exercice 2: Exploration des Seeds
Gardez le même prompt et générez 6 variations en changeant la seed. Identifiez celle qui correspond le mieux à votre vision.

### Exercice 3: Texte Créatif
Créez une image contenant un message personnalisé (ex: affiche de film, couverture de livre).

In [10]:
# =============================================================================
# 10. ESPACE D'EXERCICES
# =============================================================================

# Exercice 1: Portrait Professionnel
# Décommentez et personnalisez:

# portrait_prompt = """
# A photorealistic portrait of a professional woman in her 30s,
# wearing a navy blue blazer, confident smile,
# shot with a Canon EOS R5, 85mm f/1.4, studio lighting,
# clean white background, corporate headshot
# """
# 
# if flux:
#     images = flux.generate(portrait_prompt, width=768, height=1024, seed=42)
#     if images:
#         plt.imshow(images[0])
#         plt.axis('off')
#         plt.show()

print("📝 Espace d'exercices - Décommentez le code ci-dessus pour commencer")

📝 Espace d'exercices - Décommentez le code ci-dessus pour commencer


## 11. Récapitulatif et Points Clés

### Paramètres FLUX.1

| Paramètre | Schnell | Dev | Description |
|-----------|---------|-----|-------------|
| `num_inference_steps` | 1-4 | 20-50 | Nombre d'itérations |
| `guidance_scale` | 0.0 | 3.0-7.0 | Adhérence au prompt |
| `width/height` | 512-2048 | 512-2048 | Multiples de 8 |

### Points Forts de FLUX.1

1. **Génération de texte** exceptionnelle
2. **Rapidité** avec schnell (1-4 steps)
3. **Qualité photo-réaliste** supérieure
4. **Compréhension des prompts** avancée
5. **Support LoRA** (dev uniquement)

### Bonnes Pratiques

- Utilisez **schnell** pour le prototypage rapide
- Passez à **dev** pour la qualité finale
- Spécifiez le style photographique/artistique dans le prompt
- Pour le texte, encadrez-le avec des guillemets dans le prompt

### Ressources

- [FLUX.1 on HuggingFace](https://huggingface.co/black-forest-labs)
- [fal.ai Documentation](https://fal.ai/models/fal-ai/flux)
- Notebook suivant: **02-3-Stable-Diffusion-3-5**

In [11]:
# =============================================================================
# FIN DU NOTEBOOK
# =============================================================================

print("\n" + "="*60)
print("   ✅ Notebook FLUX.1 Advanced Generation Complété")
print("="*60)
print(f"\n📅 Terminé: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n📚 Concepts couverts:")
print("   • Architecture FLUX.1 (schnell, dev, pro)")
print("   • Client unifié (local + API)")
print("   • Analyse des inference steps")
print("   • Ratios d'aspect")
print("   • Prompt engineering avancé")
print("   • Génération de texte dans les images")
print("\n➡️  Prochain notebook: 02-3-Stable-Diffusion-3-5.ipynb")


   ✅ Notebook FLUX.1 Advanced Generation Complété

📅 Terminé: 2026-02-25 20:52:27

📚 Concepts couverts:
   • Architecture FLUX.1 (schnell, dev, pro)
   • Client unifié (local + API)
   • Analyse des inference steps
   • Ratios d'aspect
   • Prompt engineering avancé
   • Génération de texte dans les images

➡️  Prochain notebook: 02-3-Stable-Diffusion-3-5.ipynb
